# To start, download all the dependencies that are needed

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:

!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

In [4]:
import folium # map rendering library
import geocoder # import geocoder

In [5]:
import requests # library to handle requests
from urllib import request, response, error, parse
from urllib.request import urlopen

from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json # library to handle JSON files

print('Libraries imported.')

Libraries imported.


In [10]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['ON_PostalCode', 'ON_Borough', 'ON_Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"ON_PostalCode": postal_code, 
                                        "ON_Borough": borough, 
                                        "ON_Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]
toronto_data.head()

,ON_PostalCode,ON_Borough,ON_Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


In [11]:
# Get latitude & logitude from geocoder for all postcodes dataframe

latitude=[]
longitude=[]
for index,row in toronto_data.iterrows():
    lat_lng_coords = None
    #while(lat_lng_coords is None):
    g = geocoder.arcgis(row[2].format(row[0]))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
toronto_data['ON_Latitude'] =latitude
toronto_data['ON_Longitude']=longitude

toronto_data.head()

,ON_PostalCode,ON_Borough,ON_Neighborhood,ON_Latitude,ON_Longitude
0,M3A,North York,Parkwoods,19.259450,72.966880
1,M4A,North York,Victoria Village,23.083330,-74.950000
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",22.302674,114.191785
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723570,-79.437110
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government,44.388820,-79.699720


Load and explore the datasets

In [12]:
# Load the data for New york Neighboorhood
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
# newyork_data

newyork_data = newyork_data['features']
newyork_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

Tranform the data into a pandas dataframe
The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [13]:
# define the dataframe columns for Newyork

NY_column_names = ['NY_Borough', 'NY_Neighborhood', 'NY_Latitude', 'NY_Longitude'] 

# instantiate the dataframe
df_newyork = pd.DataFrame(columns=NY_column_names)

# loop through the data and fill the dataframe one row at a time.
for data in newyork_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_newyork = df_newyork.append({'NY_Borough': borough,
                                          'NY_Neighborhood': neighborhood_name,
                                          'NY_Latitude': neighborhood_lat,
                                          'NY_Longitude': neighborhood_lon}, ignore_index=True)
df_newyork.head()

,NY_Borough,NY_Neighborhood,NY_Latitude,NY_Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_newyork['NY_Borough'].unique()),
        df_newyork.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Use geopy library to get the latitude and longitude values of the Cities.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [15]:
ON_address = 'Toronto, ON'

ON_geolocator = Nominatim(user_agent="ON_explorer")
ON_location = ON_geolocator.geocode(ON_address)
ON_latitude = ON_location.latitude
ON_longitude = ON_location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(ON_latitude, ON_longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
NY_address = 'New York City, NY'

NY_geolocator = Nominatim(user_agent="NY_explorer")
NY_location = NY_geolocator.geocode(NY_address)
NY_latitude = NY_location.latitude
NY_longitude = NY_location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(NY_latitude, NY_longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [17]:
# Create a map of the City with neighborhoods superimposed on top
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[NY_latitude, NY_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_newyork['NY_Latitude'], df_newyork['NY_Longitude'], df_newyork['NY_Borough'], df_newyork['NY_Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [19]:
# Create a map of the City with neighborhoods superimposed on top
# create map of Toronto using latitude and longitude values

map_toronto = folium.Map(location=[ON_latitude, ON_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['ON_Latitude'], toronto_data['ON_Longitude'], toronto_data['ON_Borough'], toronto_data['ON_Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#4297dd',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Utilizing the Foursquare API to explore the neighborhoods and segment them.

Let's explore The popular Downtown neighborhoods in each City.
Get the neighborhood's name.

In [36]:
# First slice the original dataset and create a new dataframe for the selected neighborhoods

# Downtown Toronto
ON_neighborhood = toronto_data[toronto_data['ON_Borough'] == 'Downtown Toronto'].reset_index(drop=True)
ON_neighborhood.head(7)


,ON_PostalCode,ON_Borough,ON_Neighborhood,ON_Latitude,ON_Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",22.302674,114.191785
1,M5B,Downtown Toronto,"Garden District, Ryerson",26.149000,-81.794410
2,M5C,Downtown Toronto,St. James Town,43.669420,-79.372920
3,M5E,Downtown Toronto,Berczy Park,43.893330,-79.306930
4,M5G,Downtown Toronto,Central Bay Street,43.263700,-79.871340
5,M6G,Downtown Toronto,Christie,35.965580,-94.684570
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",-34.938360,138.561980


In [33]:

# Downtown NewYork
NY_neighborhood = df_newyork[df_newyork['NY_Borough'] == 'Brooklyn'].reset_index(drop=True)
NY_neighborhood

,NY_Borough,NY_Neighborhood,NY_Latitude,NY_Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471
5,Brooklyn,Brighton Beach,40.576825,-73.965094
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438
8,Brooklyn,Flatbush,40.636326,-73.958401
9,Brooklyn,Crown Heights,40.670829,-73.943291


Let's compare the number of neighborhoods in each of the Borough that comprise the Downtown areas of the Toronto and New York

In [30]:
ON_neighborhood.shape

(17, 5)

In [28]:
NY_neighborhood.shape

(70, 4)

We can see that New York downtown's Borough has more neigborhoods than that of Toronto
Also notice that the Downtown area of New York is a neighborhood inside of Brooklyn, while the downtown of Toronto is an entire Borough. 

In [31]:
# NY
NY_address = 'Brooklyn, NY'
NY_geolocator = Nominatim(user_agent="NY_explorer")
NY_location = NY_geolocator.geocode(NY_address)
NY_latitude = NY_location.latitude
NY_longitude = NY_location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(NY_latitude, NY_longitude))


The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [32]:
# ON
ON_address = 'Downtown Toronto, ON'
ON_geolocator = Nominatim(user_agent="ON_explorer")
ON_location = ON_geolocator.geocode(ON_address)
ON_latitude = ON_location.latitude
ON_longitude = ON_location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(ON_latitude, ON_longitude))


The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


# Define Foursquare Credentials and Version

In [34]:
CLIENT_ID = 'S30DM1QMUKVEBRGH01RJYLPCMA3E1S4DGLJAVHGUEPY1NXNQ' # your Foursquare ID
CLIENT_SECRET = '3RFQUBILDFO3DHNFQBRTUR5XHJJ53F2LEH2UT2DKDRBTPO25' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S30DM1QMUKVEBRGH01RJYLPCMA3E1S4DGLJAVHGUEPY1NXNQ
CLIENT_SECRET:3RFQUBILDFO3DHNFQBRTUR5XHJJ53F2LEH2UT2DKDRBTPO25


Let's pick the neighborhoods in our new dataframe to explore. For New York, we will select the Downtown index, while we will select the entire neighborhoods based on the fact that the entire neighborhood makes up the Downtown of Toronto

In [37]:
#Get the neighborhood's name.
# in Brooklyn 
NY_neighborhood.loc[40, 'NY_Neighborhood']


'Downtown'

In [40]:

# in Downtown Toronto
ON_neighborhood['ON_Neighborhood']

0                             Regent Park, Harbourfront
1                              Garden District, Ryerson
2                                        St. James Town
3                                           Berczy Park
4                                    Central Bay Street
5                                              Christie
6                              Richmond, Adelaide, King
7     Harbourfront East, Union Station, Toronto Islands
8              Toronto Dominion Centre, Design Exchange
9                        Commerce Court, Victoria Hotel
10                       University of Toronto, Harbord
11            Kensington Market, Chinatown, Grange Park
12    CN Tower, King and Spadina, Railway Lands, Har...
13                                             Rosedale
14                          St. James Town, Cabbagetown
15               First Canadian Place, Underground city
16                                 Church and Wellesley
Name: ON_Neighborhood, dtype: object

Let's get the geographical coordinates of the Areas of interest.

In [48]:
# New York
NY_neighborhood_latitude = NY_neighborhood.loc[40, 'NY_Latitude'] # neighborhood latitude value
NY_neighborhood_longitude = NY_neighborhood.loc[40, 'NY_Longitude'] # neighborhood longitude value

NY_neighborhood_name = NY_neighborhood.loc[40, 'NY_Neighborhood'] # neighborhood name

print('Latitude and longitude values of New York {} are {}, {}.'.format(NY_neighborhood_name, 
                                                               NY_neighborhood_latitude, 
                                                               NY_neighborhood_longitude))


Latitude and longitude values of New York Downtown are 40.69084402109802, -73.98346337431099.


# Now, let's get the top 100 venues that are within a radius of 500 meters in New York's  Downtown.

In [49]:
# let's create the GET request URL

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create New York URL
NY_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    NY_neighborhood_latitude, 
    NY_neighborhood_longitude, 
    radius, 
    LIMIT)
NY_url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=S30DM1QMUKVEBRGH01RJYLPCMA3E1S4DGLJAVHGUEPY1NXNQ&client_secret=3RFQUBILDFO3DHNFQBRTUR5XHJJ53F2LEH2UT2DKDRBTPO25&v=20180605&ll=40.69084402109802,-73.98346337431099&radius=500&limit=100'

In [50]:
results = requests.get(NY_url).json()
results

{'meta': {'code': 200, 'requestId': '5e74f5e4a2e538001b3d281c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Brooklyn',
  'headerFullLocation': 'Downtown Brooklyn, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 119,
  'suggestedBounds': {'ne': {'lat': 40.695344025598025,
    'lng': -73.97753964150077},
   'sw': {'lat': 40.68634401659802, 'lng': -73.9893871071212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5722dcad498e1dc59a10bca0',
       'name': 'Alamo Drafthouse Cinema',
       'location': {'address': '445 Albee Square West',
        'crossStreet': 'Fulton St.',
        'lat': 40.691015582921

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [51]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [55]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Alamo Drafthouse Cinema,Movie Theater,40.691016,-73.983686
1,DeKalb Market Hall,Food Court,40.691250,-73.982579
2,Trader Joe's,Grocery Store,40.691340,-73.982515
3,Daigo Hand Roll Bar,Japanese Restaurant,40.691259,-73.982603
4,Ample Hills Creamery,Ice Cream Shop,40.691291,-73.982561


In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [56]:
nearby_venues.groupby('name').count()

,categories,lat,lng
name,,,
2 Bros. Pizza,1,1,1
A Taste Of Katz's,1,1,1
Alamo Drafthouse Cinema,1,1,1
Ample Hills Creamery,1,1,1
Arepa Lady,1,1,1
BAM Harvey Theater,1,1,1
BK Jani,1,1,1
BKLYN Air,1,1,1
BRIC House,1,1,1


The results shows that all the venues in downtown New York are unqiue. This means there is no more than one venue name in the downtown. While there are 71 unique categories as shown in the unqiue categories check below.

In [58]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 71 uniques categories.


In [60]:
# Toronto
ON_neighborhood_latitude = ON_neighborhood['ON_Latitude'] # neighborhood latitude value
ON_neighborhood_longitude = ON_neighborhood['ON_Longitude'] # neighborhood longitude value

ON_neighborhood_name = ON_neighborhood['ON_Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(ON_neighborhood_name, 
                                                               ON_neighborhood_latitude, 
                                                               ON_neighborhood_longitude))


Latitude and longitude values of 0                             Regent Park, Harbourfront
1                              Garden District, Ryerson
2                                        St. James Town
3                                           Berczy Park
4                                    Central Bay Street
5                                              Christie
6                              Richmond, Adelaide, King
7     Harbourfront East, Union Station, Toronto Islands
8              Toronto Dominion Centre, Design Exchange
9                        Commerce Court, Victoria Hotel
10                       University of Toronto, Harbord
11            Kensington Market, Chinatown, Grange Park
12    CN Tower, King and Spadina, Railway Lands, Har...
13                                             Rosedale
14                          St. James Town, Cabbagetown
15               First Canadian Place, Underground city
16                                 Church and Wellesley
Name: ON_Neighb

In [61]:
# create Toronto URL
ON_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ON_neighborhood_latitude, 
    ON_neighborhood_longitude, 
    radius, 
    LIMIT)
ON_url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=S30DM1QMUKVEBRGH01RJYLPCMA3E1S4DGLJAVHGUEPY1NXNQ&client_secret=3RFQUBILDFO3DHNFQBRTUR5XHJJ53F2LEH2UT2DKDRBTPO25&v=20180605&ll=0     22.302674\n1     26.149000\n2     43.669420\n3     43.893330\n4     43.263700\n5     35.965580\n6    -34.938360\n7     22.294740\n8     43.648190\n9    -37.587369\n10    43.663200\n11    43.654320\n12    43.645437\n13    39.345750\n14    43.667600\n15    43.648360\n16    42.295656\nName: ON_Latitude, dtype: float64,0     114.191785\n1     -81.794410\n2     -79.372920\n3     -79.306930\n4     -79.871340\n5     -94.684570\n6     138.561980\n7     113.929750\n8     -79.380110\n9     144.718310\n10    -79.401290\n11    -79.401900\n12    -79.395143\n13    -76.506740\n14    -79.364190\n15    -79.381500\n16    -71.294982\nName: ON_Longitude, dtype: float64&radius=500&limit=100'

In [62]:
ON_results = requests.get(ON_url).json()
ON_results

{'meta': {'code': 400,
  'errorType': 'param_error',
  'errorDetail': 'll must be of the form XX.XX,YY.YY (received 0     22.302674\n1     26.149000\n2     43.669420\n3     43.893330\n4     43.263700\n5     35.965580\n6    -34.938360\n7     22.294740\n8     43.648190\n9    -37.587369\n10    43.663200\n11    43.654320\n12    43.645437\n13    39.345750\n14    43.667600\n15    43.648360\n16    42.295656\nName: ON_Latitude, dtype: float64,0     114.191785\n1     -81.794410\n2     -79.372920\n3     -79.306930\n4     -79.871340\n5     -94.684570\n6     138.561980\n7     113.929750\n8     -79.380110\n9     144.718310\n10    -79.401290\n11    -79.401900\n12    -79.395143\n13    -76.506740\n14    -79.364190\n15    -79.381500\n16    -71.294982\nName: ON_Longitude, dtype: float64)',
  'requestId': '5e74fe7d6001fe001c631c76'},
 'response': {}}

In [63]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
ON_venues = ON_results['response']['groups'][0]['items']
    
ON_nearby_venues = json_normalize(ON_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
ON_nearby_venues =ON_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
ON_nearby_venues['venue.categories'] = ON_nearby_venues.apply(get_category_type, axis=1)

# clean columns
ON_nearby_venues.columns = [col.split(".")[-1] for col in ON_nearby_venues.columns]

ON_nearby_venues.head()

KeyError: 'groups'

In [71]:
print('{} venues were returned by Foursquare.'.format(ON_nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [77]:
print('There are {} uniques categories.'.format(len(ON_nearby_venues['categories'].unique())))

KeyError: 'categories'